In [1]:
from math import *
from decimal import *
from copy import *
from threading import *

import random
import numpy as np

import sys
sys.path.append('..')
from game.mancala import Mancala
# from alpha_beta_pruning.alpha_beta_pruning_new import alpha_beta_pruning
from alpha_beta_pruning.alpha_beta_pruning import alpha_beta_pruning

INFINITY = 1.0e400

class Node:
    def __init__(self, side:str, move:int=None, parentNode=None, game:Mancala=None):
        # the move that got us to this node - "None" for the root node
        self.move = move
        self.side = side
        self.parentNode = parentNode
        
        self.board = game.board
        self.childNodes = []
        self.possibleMoves = game.get_valid_moves(side)
        
        self.wins = 0
        self.visits = 0
        
        # alpha represents the balanced positions function
        # beta(ni, ni') where ni' is the number of all playouts containing move i,
        # ni stands for the number of simulations for the node considered after the i-th move
        # the value of alpha is chosen empirically
        self.alpha = 0.15
        
        # evalue represents the ratio of wi' to ni', 
        # where wi' stands for the number of won playouts containing move i
        self.evalue = 0.0
        
        # whether or not the next move is still the current player
        self.again = False
        
        # the evaluation score of the move
        self.rvalue = 0.0
        
        # current node is the max, from the current player's view,
        # will be changed if the next player is no longer current player
        self.minimax = True
        
        
        
    def addChild(self, side, move, game:Mancala, evalue, rvalue, again):
        
        node = Node(side, move, parentNode=self, game=game)
        
        node.evalue = evalue
        node.rvalue = rvalue
        node.again = again
        
        if again:
            node.minimax = self.minimax
        else:
            node.minimax = not self.minimax
        
        self.possibleMoves.remove(move)
        self.childNodes.append(node)
        return node
    
    def get_opponent_side(self):
        return 'north' if self.side == 'south' else 'south'
    
    def update(self, win, lose):
        visits = win + lose
        self.wins += win
        self.visits += visits
    
    def __str__(self):
        return "[Move:" + str(self.move) + \
                "Wins/Visits:" + str(self.wins) + "/" + str(self.visits) + \
                "Side:" + side + \
                "]\n"
    
    def TreeToString(self, indent, level):
        if level == 0:
            return " "
        s = self.indentToString(indent) + str(self)
        for child in self.childNodes:
            s += child.TreeToString(indent+1, level-1)
        return s
    
    def indentToString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s
    
#     def ChildrenToString(self):
#         s = ""
#         for child in self.childNodes:
#             s += str(child) + "\n"
#         return s
    
    def UCTselectChild(self):
        # UCB1 formula
        formula = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits)
        
        # UCB1 RAVE formula
#         formula = lambda c: (1-c.alpha)*(c.wins/c.visits)+ c.alpha*c.evalue + sqrt(2*log(self.visits)/c.visits)
        
        result = sorted(self.childNodes, key=formula)[-1]
        return result
        

In [2]:
def winlose(side, game:Mancala):
    # print the heuristics of the current side
    
    s1 =0.0
    s2 =0.0
    
    # current player wins
    if game.has_over_half_stones(side):
        return INFINITY
    
    # current player loses
    if game.has_over_half_stones(game.get_opponent_side(side)):
        return -INFINITY
    
    # score of current player
    s1 = game.board[game.get_store(side)]

    # score of other player
    s2 = game.board[game.get_store(game.get_opponent_side(side))]
    
    
    # total score
    total = float(game.n_stones * game.n_holes * 2)
    
    return ((s1 - s2) / total + sqrt(s1) / sqrt(total)) /2

def score(side, game:Mancala):
    # print the current score of a side, which is from -1 to 1(including -1 and 1)
    
    
    s1 = 0.0 ### all scores from base and this side's stones
    s2 = 0.0 ### all scores from opposite base and opposite stones
    ######################################################################################
    ### should set weight to scores for different side ###################################
    ######################################################################################
    s1 += np.sum(game.board[game.get_start_hole(side):game.get_store(side)])
    s1 += game.board[game.get_store(side)]
    
    if game.has_over_half_stones(side):
        return 1
    
    s2 += np.sum(game.board[game.get_start_hole(game.get_opponent_side(side)):game.get_store(game.get_opponent_side(side))])
    s2 += game.board[game.get_store(game.get_opponent_side(side))]
    
    
    if game.has_over_half_stones(game.get_opponent_side(side)):
        return -1 
    
    total = float(game.n_stones * game.n_holes * 2)
    return ((s1-s2) / total + sqrt(s1)/sqrt(total/2)) /2

def catchMove(side, game:Mancala, move):
    # whether exists a move, that can catch the stones opposite to an empty hole
    # returns a boolean of availability of catching and the move to catch it
    
    ### note: if there are multiple catchMove exists, function will take the first detected one.
    ### if there is no possible catch, return False, -1
    ### example output:
    ###   
    ###  3  [ 10  10  10  10   2   0   1]
    ###     [  1  10   8   8   8   8   8]  1
    ###     Traps for:south hole=6 oppoHole=1
    ###     True, 0
    ###     
    ### this means there is a catchMove for north, to move the 'oppoHole' hole
    ### so south now needs to move the 'hole'

    
    ### self holes, normally is game.board[0:7]
    selfHoles = game.board[game.get_start_hole(side):game.get_store(side)]
#     print(selfHoles)
    ### opponent's holes, normally is game.board[8:15]
    oppoHoles = game.board[game.get_start_hole(game.get_opponent_side(side)):game.get_store(game.get_opponent_side(side))]
#     print(oppoHoles)
    
    if selfHoles[move-1] > 1 and oppoHoles[len(oppoHoles)-move] == 0:
#         print(True)
        for i in range(len(oppoHoles)):
        
            if oppoHoles[i] == 0:
                continue
            
            ### first array
            if oppoHoles[i] + i <= 6:
                if oppoHoles[i] + i == len(oppoHoles)-move:
                    print("Traps for:"+str(side)+" hole="+str(move)+" oppoHole="+str(i+1))
                    return True, i
            ### second array
            elif oppoHoles[i] + i >= 15:
                if oppoHoles[i] + i - 15 == len(oppoHoles)-move:
                    print("Traps for:"+str(side)+" hole="+str(move)+" oppoHole="+str(i+1))
                    return True, i
    return False, -1

def get_again(game:Mancala, side):
    return True if game.next_player == side else False

def get_opponent_side(side):
    return 'north' if side == 'south' else 'south'
# game = Mancala()
# game.step('north', 3)
# print(game)
# game.step('north', 1)
# print(game)
# game.step('south', 1)
# print(game)
# game.step('north', 2)
# print(game)

# result = score('south', game)
# print(result)

# a = [4] * 6
# a = [1,2,3,4,5,6]
# b = [7,8,9,10,11,12]
# print(a)
# print(b)
# print(a[6-1])
# print(b[1-1])

# catch, move = catchMove('south', game, 6)
# print(catch)
# print(move)

In [3]:
def UCT(rootGame:Mancala, iterMax:int, side:str):
    
    ### rootgame: the game Mancala
    ### iterMax: the maximum round of iteration of tree search, minimum number is 5000 iterations
    ### side: the player(south or north)
    
    ### Conduct a UCT search for itermax iterations starting from rootstate.
    ### Return the best move from the rootstate.
    
    
    rootNode = Node(game = rootGame, side = side)
    rootPlayer = deepcopy(side)
    
    ### self holes, normally is game.board[8:15]
    selfHoles = game.board[game.get_start_hole(side):game.get_store(side)]
#     print(selfHoles)
    ### opponent's holes, normally is game.board[0:7]
    oppoHoles = game.board[game.get_start_hole(game.get_opponent_side(side)):game.get_store(game.get_opponent_side(side))]
#     print(oppoHoles)

#     print('UCT Starts!!!!!!!!!!')
    print(rootGame)
    
    for i in range(iterMax):
        
        if i % 5000 == 0:
            print("Processing :"+str(i)+"/"+str(itermax))
        
        node = rootNode
        game = deepcopy(rootGame)
        
        
        ### return If there is a win move found only when root node is fully exploited
        ### assume the mamimum-visits is 100, as there are only 98 stones
        if rootNode.childNodes != [] and node.childNodes != [] and rootnode.visits == 100:
            
            for child in rootNode.childNodes:
                
                if child.rvalue == 1:
                    return child.move
                
                if child.again:
                    move, _ = alpha_beta_pruning(rootGame, rootPlayer, depth=3)
                    print("Again child.Move="+str(child.move)+" Alpha-Beta Depth=3 Move="+str(move))
                    return move
                
                catch_possible, catch_move = catchMove(rootPlayer, rootGame, child.move)
                ### if we can catch a move, then we do that move
                if catch_possible: return child.move
                
                catch_possible, catch_move = catchMove(get_opponent_side(rootPlayer), rootGame, child.move)
                ### if opponent can catch our move, then we do the move which prevents that
                if catch_possible: return catch_move
        
        
        
        ##### SELECT #####
        while node.possibleMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTselectChild()
            game.step(node.side, node.move)
#             print("Select Start Player:"+str(node.player.num)+" Move:"+str(node.move))
        
        
        
        ##### Expand #####
        if node.possibleMoves != []:
            
            move = random.choice(node.possibleMoves)
            game.step(node.side, move)
            again = get_again(game, node.side)
#             print("Expand make move Player:"+str(node.player.num)+" Move:"+str(m))

            if again:
                nodePlayer = node.side
            else:
                nodePlayer = node.get_opponent_side(side)
#             print("Expand AddChild Player:"+str(nodeplayer)+" Move:"+str(m))
            
            evalue = score(rootPlayer, game)
            rvalue = winlose(rootPlayer, game)
            
            node = node.addChild(nodePlayer, move, game, evalue, rvalue, again)
        
        
        ##### RollOut #####
        while not game.game_over:
            
            if game.has_over_half_stones(side) or game.has_over_half_stones(game.get_opponent_side(side)):
                break
            
            ### temporary using minimax
            move, _ = alpha_beta_pruning(game, nodePlayer, depth=1)
            
            game.step(nodePlayer, move)
            again = get_again(game, nodePlayer)
            if not again:
                nodePlayer = get_opponent_side(nodePlayer)
        
        ##### BackPropagate #####
        